In [3]:
import numpy as np
import pandas as pd
import tweepy
import sys
import json
import time


class TwitterStreamListener(tweepy.StreamListener):
    
    def __init__(self, timeout=None):
        super(TwitterStreamListener, self).__init__()
        self.timeout = timeout
        self.start_time = time.time()
    
    def on_status(self, status):
        
        if self.timeout:
            if time.time()-self.start_time < self.timeout:          
                print(f'Tweet id: ',status.id,'\n',
                     'Date: ',status.created_at,'\n',
                     'Text: ',status.text,'\n')
                return True
            else:
                return False
        else:
            print(f'Tweet id: ',status.id,'\n',
                     'Date: ',status.created_at,'\n',
                     'Text: ',status.text,'\n')
    
    def on_error(self, status_code):
        print(status_code)

        
class Twitter_crawler:
    
    def __init__(self, API_key, API_secret_key, access_token, access_token_secret, StreamListener=None):   
        
        self.consumer_key = API_key
        self.consumer_secret = API_secret_key
        self.access_token = access_token
        self.access_token_secret = access_token_secret
        self.auth = None
        self.api = None
        self.stream = None
        
    def connect(self):
        
        self.auth = tweepy.OAuthHandler(self.consumer_key, self.consumer_secret)
        self.auth.set_access_token(self.access_token, self.access_token_secret)
        self.api = tweepy.API(self.auth)
        
    def stream_twits(self, topic, timeout=None):
        
        try:
            self.stream = tweepy.Stream(auth = self.api.auth, listener = TwitterStreamListener(timeout))
            self.stream.filter(track=[topic])
                
        except Exception as excp:
            print('Streaming failed, make sure that you are connected to the Twitter API and/or a stream listener is provided')
            raise excp
            
    def get_data(self, topic, nb_tweets, date_start='2018-11-16'):
        
        try:
            alltweets = {'tweet_id':[], 'date':[], 'text':[]}
            tweets = tweepy.Cursor(self.api.search, q=topic, lang="en", since=date_start).items(nb_tweets)
            for t in tweets:
                alltweets['tweet_id'].append(t._json['id_str'])
                alltweets['date'].append(t._json['created_at'])
                alltweets['text'].append(t._json['text'])
            df = pd.DataFrame(alltweets, columns=alltweets.keys())
            return df
            
        except Exception as excp:
            print('Data extraction failed, make sure that you are connected to the Twitter API')
            raise excp
                  
    def get_user_data(self, name, nb_tweets):
    
        try:
            alltweets = []
            new_tweet = self.api.user_timeline(screen_name=name, count=nb_tweets)
            alltweets.extend(new_tweet)
            outtweets = np.array([[tweet.id_str, tweet.created_at, tweet.text.encode("utf-8")] for tweet in alltweets])
            df = pd.DataFrame(outtweets, columns=['tweet_id', 'date', 'text'])
            return df
            
        except Exception as excp:
            print(f'data extraction of user "{name}" failed, make sure that you are connected to the Twitter API')
            raise excp

crawlerTwitter = Twitter_crawler("Kbb5IgwOWNOGWGnR3rzqNqfCC", "jIftp2C79iKyRm1dWRVmiQ1oeD401XD2aKFg7gaUFk0KaDsVcd",
                                    "1249135289844539392-uYZjVAPrqXT1AhYnSe4eJ8VNCyu5cD", 
                                     "ugwFhC517G2r3LPq8pZZLhb9SoAVmh9PAvF67SEAZxnQI")

crawlerTwitter.connect()
#df = crawlerTwitter.get_user_data('omaross', 1000)
#crawlerTwitter.stream_twits('bitcoin')
#tweets_bitcoin = crawlerTwitter.get_data('bitcoin', 500)



In [234]:
df_twits = crawlerTwitter.get_data('bitcoin', 500)
df_twits

,tweet_id,date,text
0,1386473715110719488,Mon Apr 26 00:14:27 +0000 2021,Which comes first? #bitcoin
1,1386473714968154113,Mon Apr 26 00:14:27 +0000 2021,RT @limitstream: 🎁 $50 Bitcoin Giveaway 🎁\n\n✅...
2,1386473714716401669,Mon Apr 26 00:14:27 +0000 2021,RT @saifedean: This sociopath hates bitcoin be...
3,1386473711444783109,Mon Apr 26 00:14:26 +0000 2021,RT @Excellion: Asia just woke up and bought th...
4,1386473709427331073,Mon Apr 26 00:14:26 +0000 2021,"RT @flurbnb: $500 reroll, sponsor just disappe..."
...,...,...,...
495,1386472588440969217,Mon Apr 26 00:09:59 +0000 2021,@JasonYanowitz This little dip you are freakin...
496,1386472585622265857,Mon Apr 26 00:09:58 +0000 2021,RT @WClementeIII: On-Chain Volume support at $...
497,1386472585332940800,Mon Apr 26 00:09:58 +0000 2021,@chrishlad Great stuff!\n\nBuy #Bitcoin #Ethereum
498,1386472582094983170,Mon Apr 26 00:09:57 +0000 2021,RT @Excellion: Reminder: We are still in the m...


In [235]:
df_musk_twits = crawlerTwitter.get_user_data('elonmusk', 200)
df_musk_twits

,tweet_id,date,text
0,1386405029523845125,2021-04-25 19:41:31,b'@BadAstronomer So awesome!'
1,1386158212970340354,2021-04-25 03:20:46,b'Let\xe2\x80\x99s find out just how live Satu...
2,1386155357739921412,2021-04-25 03:09:25,b'Am hosting SNL on May 8'
3,1386148075434123264,2021-04-25 02:40:29,b'@Tesmanian_com Yup'
4,1385887339742998528,2021-04-24 09:24:24,b'RT @SpaceX: Docking confirmed \xe2\x80\x93 s...
...,...,...,...
195,1378836600881352706,2021-04-04 22:27:17,b'@tobyliiiiiiiiii @katlinegrey I recommend pr...
196,1378836183745191936,2021-04-04 22:25:38,b'@PPathole @katlinegrey For practical purpose...
197,1378775743115694085,2021-04-04 18:25:28,"b'@katlinegrey This a good path, but I recomme..."
198,1378661987324993542,2021-04-04 10:53:26,b'Urgent need to build more housing in greater...


In [9]:
crawlerTwitter.stream_twits('bitcoin')

Tweet id:  1386772221612498945 
 Date:  2021-04-26 20:00:37 
 Text:  RT @BitcoinMagazine: JPMorgan Reportedly Planning To Offer A Bitcoin Fund.

“The JPMorgan bitcoin funding could roll out as soon as this su… 

Tweet id:  1386772221838782466 
 Date:  2021-04-26 20:00:37 
 Text:  Use Coinbase it allows you to trade crypto with no hassle! No ID needed for sending and receiving only buying and s… https://t.co/vD7fpIv25L 

Tweet id:  1386772222681911300 
 Date:  2021-04-26 20:00:37 
 Text:  RT @RemitanoL: #remitano #remitanoexchange #bitcoin #invest #coininvest #cryptocurrency #dolarparalelo #estiempodebitcoin #noticias #econom… 

Tweet id:  1386772222774231040 
 Date:  2021-04-26 20:00:37 
 Text:  La bougie d'avril sera en verte bisous
#Bitcoin 

Tweet id:  1386772225307537409 
 Date:  2021-04-26 20:00:38 
 Text:  RT @bricepromos: It's crazy how $VRX here is like $BNB at 1$ !

If you missed all the big crypto bullruns,
Don't miss this next!

$STEP $RA… 

Tweet id:  1386772228180692993 
 

Tweet id:  1386772270547230725 
 Date:  2021-04-26 20:00:48 
 Text:  @mcuban @BitPay Wow good news. Doge will replace Bitcoin. 

Tweet id:  1386772270996025344 
 Date:  2021-04-26 20:00:49 
 Text:  RT @CoinATMRadar: Check this out! Just installed @genesiscoin #bitcoin ATM at  Shell Leesburg Pike in #FallsChurch 🇺🇸. Operated by @GetCoin… 

Tweet id:  1386772274397777920 
 Date:  2021-04-26 20:00:49 
 Text:  RT @ICOAffiliates: 15$ to one person who likes &amp; rts this post + add #CBMwin to there dio also tag 3 or more friends!
Ends in 3hrs good luc… 

Tweet id:  1386772275408605190 
 Date:  2021-04-26 20:00:50 
 Text:  RT @DarrenJBeattie: This is why the square faced Sir yes SIR!!! Chicom nonsense can be so dangerous

Using Chicom boogey man to blackmail y… 

Tweet id:  1386772275605684225 
 Date:  2021-04-26 20:00:50 
 Text:  viajaria a el primer medio petro q dio el gobierno y viajaria mas atras por 2 bitcoin q dio el creador de atari q s… https://t.co/i8YrKgsMgc 

Tweet id:  13867722

Tweet id:  1386772311710310406 
 Date:  2021-04-26 20:00:58 
 Text:  RT @BTCization: Who’s announcing a bitcoin position this week? https://t.co/uI5jm8DE05 

Tweet id:  1386772312012296206 
 Date:  2021-04-26 20:00:58 
 Text:  🧐 $5M 

Tweet id:  1386772312519806977 
 Date:  2021-04-26 20:00:58 
 Text:  RT @business: JPMorgan is preparing to offer a Bitcoin fund to wealthy clients https://t.co/9nruGVjSun 

Tweet id:  1386772313266221056 
 Date:  2021-04-26 20:00:59 
 Text:  Exactly 😂😂 

Tweet id:  1386772315191406592 
 Date:  2021-04-26 20:00:59 
 Text:  I want to participate in PlumCake Airdrop
@MdAbdul20018456
@MdAbdulHussen1

BSC address:0x0Db79789C778a685D2fA91dFb… https://t.co/dACn1HAhdV 

Tweet id:  1386772316147920898 
 Date:  2021-04-26 20:00:59 
 Text:  I've just predicted the future of Bitcoin! You should too :) Use my reference code 52Y4I5Y to signup at… https://t.co/80neLjwuQw 

Tweet id:  1386772316588216321 
 Date:  2021-04-26 20:00:59 
 Text:  RT @pete_rizzo_: 🚨 10 years 

Tweet id:  1386772342039269381 
 Date:  2021-04-26 20:01:05 
 Text:  RT @DoentesPFutebol: Quer ganhar dinheiro trabalhando em casa? Só usando o celular? 100% seguro e confiável? Sem cair em pirâmide?

É fácil… 

Tweet id:  1386772342643232769 
 Date:  2021-04-26 20:01:06 
 Text:  RT @lkendrick: More players &amp; employees will be requesting payment in Bitcoin (BTC), Ethereum (ETH) and #Dogecoin (Doge) too. 🐕 🚀

It's hap… 

Tweet id:  1386772342400036866 
 Date:  2021-04-26 20:01:06 
 Text:  RT @100trillionUSD: After this 2020 Vitalik tweet (BTC $9K) 2 things happened: 
1) BTC price increased 6x to $54K (exactly like S2F model p… 

Tweet id:  1386772342035070977 
 Date:  2021-04-26 20:01:05 
 Text:  #JPMorgan will allow its clients to invest in the #Bitcoin Fund for the first time. The #bitcoins fund could be imp… https://t.co/0wWVq7n5Fz 

Tweet id:  1386772342974697472 
 Date:  2021-04-26 20:01:06 
 Text:  RT @renprotocol: The long-awaited @BadgerDAO RenVM integration is now live! Bri

Tweet id:  1386772384724799489 
 Date:  2021-04-26 20:01:16 
 Text:  "I want to participate in PlumCake Airdrop"
Also leave your BSC address, tag 2 friends &amp; hashtags @plumcakefinance… https://t.co/tpnqDTWSeZ 

Tweet id:  1386772385588596738 
 Date:  2021-04-26 20:01:16 
 Text:  RT @a3voices: #Algorand is the quiet genius in class who everyone ignores.

#Cardano is like the trustworthy, popular, and average IQ frien… 

Tweet id:  1386772386716991491 
 Date:  2021-04-26 20:01:16 
 Text:  https://t.co/YBTQUAcnsd
#xauusd #xagtryg #xagusd #bist100  #xautry  #ons #GRAMALTIN  #gümüş #usdtry #dolartl… https://t.co/TcYgauXd43 

Tweet id:  1386772387870306307 
 Date:  2021-04-26 20:01:16 
 Text:  RT @mcclure111: - Paypal and Venmo let you keep a crypto wallet
- You're allowed to pay for USD transactions with the crypto in your wallet… 

Tweet id:  1386772387933347840 
 Date:  2021-04-26 20:01:16 
 Text:  That's why Bitcoin exists to begin with.
Because Americans fxcked up 

Tweet id:  13867

KeyboardInterrupt: 